In [ ]:
import open3d as o3d
import open3d.visualization.gui as gui
import open3d.visualization as vis
import open3d.visualization.rendering as rendering
import threading
import numpy as np
# from open3d.web_visualizer import _global_jupyter_loop

In [ ]:
class _AsyncEventLoop:

    class _Task:
        _g_next_id = 0

        def __init__(self, f):
            self.task_id = self._g_next_id
            self.func = f
            _AsyncEventLoop._Task._g_next_id += 1

    def __init__(self):
        # TODO: find a better solution. Currently py::print requires GIL which
        # causes deadlock when AsyncEventLoop is used. By calling
        # reset_print_function(), all C++ prints will be directed to the
        # terminal while python print will still remain in the cell.
        o3d.utility.reset_print_function()
        self._lock = threading.Lock()
        self._run_queue = []
        self._return_vals = {}
        self._started = False
        self.start()

    def start(self):
        if not self._started:
            self._thread = threading.Thread(target=self._thread_main)
            self._thread.start()
            self._started = True

    def run_sync(self, f):
        with self._lock:
            task = _AsyncEventLoop._Task(f)
            self._run_queue.append(task)

        while True:
            with self._lock:
                if task.task_id in self._return_vals:
                    return self._return_vals[task.task_id]

    def _thread_main(self):
        app = gui.Application.instance
        app.initialize()

        done = False
        while not done:
            with self._lock:
                for task in self._run_queue:
                    retval = task.func()
                    self._return_vals[task.task_id] = retval
                self._run_queue.clear()

            done = not app.run_one_tick()


# The _AsyncEventLoop class shall only be used to create a singleton instance.
# There are different ways to achieve this, here we use the module as a holder
# for singleton variables, see: https://stackoverflow.com/a/31887/1255535.
_async_event_loop = _AsyncEventLoop()

In [ ]:
from functools import partial
def web_draw(geometry):
    
    def add_window(geo):
        app = gui.Application.instance
        window = o3d.visualization.O3DVisualizer("Open3D", 640, 480)
        window.add_geometry("Torus", geo)
        window.reset_camera_to_default()
        app.add_window(window)
        return window.uid
        
    uid = _async_event_loop.run_sync(partial(add_window, geometry))
    print(f"Newly add Window: {uid}")
    visualizer = o3d.WebVisualizer(window_uid=uid)
    visualizer.show()

In [ ]:
def web_draw(geometry=None,
             title="Open3D",
             width=640,
             height=480,
             actions=None,
             lookat=None,
             eye=None,
             up=None,
             field_of_view=60.0,
             bg_color=(1.0, 1.0, 1.0, 1.0),
             bg_image=None,
             show_ui=None,
             point_size=None,
             animation_time_step=1.0,
             animation_duration=None,
             rpc_interface=False,
             on_init=None,
             on_animation_frame=None,
             on_animation_tick=None):
    
    def _web_draw(geometry,
                  title,
                  width,
                  height,
                  actions,
                  lookat,
                  eye,
                  up,
                  field_of_view,
                  bg_color,
                  bg_image,
                  show_ui,
                  point_size,
                  animation_time_step,
                  animation_duration,
                  rpc_interface,
                  on_init,
                  on_animation_frame,
                  on_animation_tick):
        gui.Application.instance.initialize()
        w = o3d.visualization.O3DVisualizer(title, width, height)
        w.set_background(bg_color, bg_image)

        if actions is not None:
            for a in actions:
                w.add_action(a[0], a[1])

        if point_size is not None:
            w.point_size = point_size

        def add(g, n):
            if isinstance(g, dict):
                w.add_geometry(g)
            else:
                w.add_geometry("Object " + str(n), g)

        n = 1
        if isinstance(geometry, list):
            for g in geometry:
                add(g, n)
                n += 1
        elif geometry is not None:
            add(geometry, n)

        w.reset_camera_to_default()  # make sure far/near get setup nicely
        if lookat is not None and eye is not None and up is not None:
            w.setup_camera(field_of_view, lookat, eye, up)

        w.animation_time_step = animation_time_step
        if animation_duration is not None:
            w.animation_duration = animation_duration

        if show_ui is not None:
            w.show_settings = show_ui

        if rpc_interface:
            w.start_rpc_interface(address="tcp://127.0.0.1:51454", timeout=10000)

            def stop_rpc():
                w.stop_rpc_interface()
                return True

            w.set_on_close(stop_rpc)

        if on_init is not None:
            on_init(w)
        if on_animation_frame is not None:
            w.set_on_animation_frame(on_animation_frame)
        if on_animation_tick is not None:
            w.set_on_animation_tick(on_animation_tick)

        gui.Application.instance.add_window(w)
        return w.uid

    uid = _async_event_loop.run_sync(partial(_web_draw, geometry = geometry,
                                                        title = title,
                                                        width = width,
                                                        height = height,
                                                        actions = actions,
                                                        lookat = lookat,
                                                        eye = eye,
                                                        up = up,
                                                        field_of_view = field_of_view,
                                                        bg_color = bg_color,
                                                        bg_image = bg_image,
                                                        show_ui = show_ui,
                                                        point_size = point_size,
                                                        animation_time_step = animation_time_step,
                                                        animation_duration = animation_duration,
                                                        rpc_interface = rpc_interface,
                                                        on_init = on_init,
                                                        on_animation_frame = on_animation_frame,
                                                        on_animation_tick = on_animation_tick))
    print(f"Newly add Window: {uid}")
    visualizer = o3d.WebVisualizer(window_uid=uid)
    visualizer.show()

In [ ]:
cube_red = o3d.geometry.TriangleMesh.create_box(1, 2, 4)
cube_red.compute_vertex_normals()
cube_red.paint_uniform_color((1.0, 0.0, 0.0))
web_draw(geometry=cube_red)

In [ ]:
cube_blue = o3d.geometry.TriangleMesh.create_box(1, 2, 4)
cube_blue.compute_vertex_normals()
cube_blue.paint_uniform_color((0.0, 0.0, 1.0))
web_draw(geometry=cube_blue)